---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
print(train_data.shape)
train_data.head()

(250306, 34)


/home/sid/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [4]:
train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)].shape

(159880, 34)

In [5]:
test_data = pd.read_csv('test.csv')
test_data.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [6]:
test_data.shape, test_data[test_data['city']=='DETROIT'].shape

((61001, 27), (26358, 27))

In [7]:
address =  pd.read_csv('addresses.csv')
address.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [8]:
latlons = pd.read_csv('latlons.csv')
latlons.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [9]:
address = address.set_index('address').join(latlons.set_index('address'), how='left')
address.head()

,ticket_id,lat,lon
address,,,
"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
"-15126 harper, Detroit MI",344821,42.406402,-82.957525
"0 10th st, Detroit MI",24928,42.325689,-83.064330
"0 10th st, Detroit MI",71887,42.325689,-83.064330


In [10]:
train_data = train_data.set_index('ticket_id').join(address.set_index('ticket_id'))
train_data.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,42.326937,-83.135118
22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.380516,-83.096069
22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,MI,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.380570,-83.095919
22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,MI,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.145257,-83.208233


In [11]:
test_data = test_data.set_index('ticket_id').join(address.set_index('ticket_id'))
test_data.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.426239,-83.238259
285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.309661,-83.122426
285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.308830,-83.121116


In [12]:
train_data[train_data['late_fee']!=10].shape

(237790, 35)

In [13]:
train_data = train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)]

In [14]:
train_data.shape

(159880, 35)

In [15]:
len(train_data['violation_code'].unique())

189

In [16]:
len(train_data['city'].unique())

4093

In [17]:
len(train_data['state'].unique())

60

In [18]:
len(train_data['agency_name'].unique())

5

In [19]:
len(test_data['agency_name'].unique())

3

In [20]:
len(train_data['disposition'].unique())

4

In [21]:
train_data['ticket_issued_date'].head()

ticket_id
22056    2004-03-16 11:40:00
27586    2004-04-23 12:30:00
22046    2004-05-01 11:50:00
18738    2004-06-14 14:15:00
18735    2004-06-16 12:30:00
Name: ticket_issued_date, dtype: object

In [22]:
train_data[train_data['ticket_issued_date'].isnull()]

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,


In [23]:
train_data = train_data[~train_data['hearing_date'].isnull()]

In [24]:
len(test_data[test_data['hearing_date'].isnull()])

2197

In [25]:
len(test_data[test_data['ticket_issued_date'].isnull()])

0

In [26]:
train_data['hearing_date'].head()

ticket_id
22056    2005-03-21 10:30:00
27586    2005-05-06 13:30:00
22046    2005-03-21 10:30:00
18738    2005-02-22 15:00:00
18735    2005-02-22 15:00:00
Name: hearing_date, dtype: object

In [27]:
from datetime import datetime
def time_gap(hearing_date_str, ticket_issued_date_str):
    if not hearing_date_str: return 73
    hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
    ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S")
    gap = hearing_date - ticket_issued_date
    return gap.days

In [28]:
gap = datetime.strptime("2005-02-22 15:00:00", "%Y-%m-%d %H:%M:%S") - datetime.strptime("2004-06-16 12:30:00", "%Y-%m-%d %H:%M:%S")
gap.days

251

In [29]:
train_data['time_gap'] = train_data.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)

In [30]:
train_data['time_gap'].mean()

72.64741032113396

In [31]:
feature_to_be_splitted = ['agency_name', 'state', 'disposition']

In [32]:
'balance_due' in train_data

True

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.tree import DecisionTreeClassifier
    from datetime import datetime
    def time_gap(hearing_date_str, ticket_issued_date_str):
        if not hearing_date_str or type(hearing_date_str)!=str: return 73
        hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
        ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S")
        gap = hearing_date - ticket_issued_date
        return gap.days
    train_data = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_data = pd.read_csv('test.csv')
    train_data = train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)]
    address =  pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    address = address.set_index('address').join(latlons.set_index('address'), how='left')
    train_data = train_data.set_index('ticket_id').join(address.set_index('ticket_id'))
    test_data = test_data.set_index('ticket_id').join(address.set_index('ticket_id'))
    train_data = train_data[~train_data['hearing_date'].isnull()]
    train_data['time_gap'] = train_data.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)
    test_data['time_gap'] = test_data.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)
    feature_to_be_splitted = ['agency_name', 'state', 'disposition']
    train_data.lat.fillna(method='pad', inplace=True)
    train_data.lon.fillna(method='pad', inplace=True)
    train_data.state.fillna(method='pad', inplace=True)

    test_data.lat.fillna(method='pad', inplace=True)
    test_data.lon.fillna(method='pad', inplace=True)
    test_data.state.fillna(method='pad', inplace=True)
    train_data = pd.get_dummies(train_data, columns=feature_to_be_splitted)
    test_data = pd.get_dummies(test_data, columns=feature_to_be_splitted)
    list_to_remove_train = [
        'balance_due',
        'collection_status',
        'compliance_detail',
        'payment_amount',
        'payment_date',
        'payment_status'
    ]
    list_to_remove_all = ['fine_amount', 'violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code',
                          'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
    train_data.drop(list_to_remove_train, axis=1, inplace=True)
    train_data.drop(list_to_remove_all, axis=1, inplace=True)
    test_data.drop(list_to_remove_all, axis=1, inplace=True)
    train_features = train_data.columns.drop('compliance')
    train_features_set = set(train_features)
    
    for feature in set(train_features):
        if feature not in test_data:
            train_features_set.remove(feature)
    train_features = list(train_features_set)
    
    X_train = train_data[train_features]
    y_train = train_data.compliance
    X_test = test_data[train_features]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = MLPClassifier(hidden_layer_sizes = [100, 10], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
#     clf = DecisionTreeClassifier()
    clf.fit(X_train_scaled, y_train)

    test_proba = clf.predict_proba(X_test_scaled)[:,1]

    
    test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    test_df['compliance'] = test_proba
    test_df.set_index('ticket_id', inplace=True)
    
    return test_df.compliance

In [2]:
predictions = blight_model()
predictions

D:\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


ticket_id
284932    0.041829
285362    0.015092
285361    0.049507
285338    0.060982
285346    0.083708
285345    0.061183
285347    0.097681
285342    0.627011
285530    0.016614
284989    0.063688
285344    0.095922
285343    0.016553
285340    0.016677
285341    0.097692
285349    0.055556
285348    0.052894
284991    0.047615
285532    0.027981
285406    0.020785
285001    0.017929
285006    0.006529
285405    0.051131
285337    0.018696
285496    0.072235
285497    0.044810
285378    0.050607
285589    0.053738
285585    0.037409
285501    0.062522
285581    0.014922
            ...   
376367    0.007117
376366    0.028148
376362    0.209954
376363    0.238736
376365    0.007117
376364    0.028148
376228    0.037890
376265    0.027437
376286    0.225193
376320    0.034755
376314    0.035267
376327    0.305467
376385    0.270971
376435    0.581721
376370    0.523673
376434    0.050636
376459    0.058609
376478    0.001490
376473    0.034913
376484    0.031730
376482    0.020713
37

In [1]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime
def time_gap(hearing_date_str, ticket_issued_date_str):
    if not hearing_date_str or type(hearing_date_str)!=str: return 73
    hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
    ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S")
    gap = hearing_date - ticket_issued_date
    return gap.days
train_data = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
test_data = pd.read_csv('test.csv')
train_data = train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)]
address =  pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')
address = address.set_index('address').join(latlons.set_index('address'), how='left')
train_data = train_data.set_index('ticket_id').join(address.set_index('ticket_id'))
test_data = test_data.set_index('ticket_id').join(address.set_index('ticket_id'))
train_data = train_data[~train_data['hearing_date'].isnull()]
train_data['time_gap'] = train_data.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)
test_data['time_gap'] = test_data.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)
feature_to_be_splitted = ['agency_name', 'state', 'disposition']
train_data.lat.fillna(method='pad', inplace=True)
train_data.lon.fillna(method='pad', inplace=True)
train_data.state.fillna(method='pad', inplace=True)

test_data.lat.fillna(method='pad', inplace=True)
test_data.lon.fillna(method='pad', inplace=True)
test_data.state.fillna(method='pad', inplace=True)
train_data = pd.get_dummies(train_data, columns=feature_to_be_splitted)
test_data = pd.get_dummies(test_data, columns=feature_to_be_splitted)
list_to_remove_train = [
    'balance_due',
    'collection_status',
    'compliance_detail',
    'payment_amount',
    'payment_date',
    'payment_status'
]
list_to_remove_all = ['fine_amount', 'violator_name', 'zip_code', 'country', 'city',
                      'inspector_name', 'violation_street_number', 'violation_street_name',
                      'violation_zip_code', 'violation_description',
                      'mailing_address_str_number', 'mailing_address_str_name',
                      'non_us_str_code',
                      'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
train_data.drop(list_to_remove_train, axis=1, inplace=True)
train_data.drop(list_to_remove_all, axis=1, inplace=True)
test_data.drop(list_to_remove_all, axis=1, inplace=True)
train_features = train_data.columns.drop('compliance')
train_features_set = set(train_features)

for feature in set(train_features):
    if feature not in test_data:
        train_features_set.remove(feature)
train_features = list(train_features_set)

X_train = train_data[train_features]
y_train = train_data.compliance
X_test = test_data[train_features]

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

D:\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
X_test

,late_fee,state_PR,state_HI,state_GA,state_PA,state_MD,state_UT,state_TX,state_DC,lat,...,state_IA,disposition_Responsible by Default,state_DE,state_MA,state_OH,agency_name_Detroit Police Department,agency_name_Department of Public Works,lon,state_AL,state_ID
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,20.0,0,0,0,0,0,0,0,0,42.407581,...,0,1,0,0,0,0,1,-82.986642,0,0
285362,100.0,0,0,0,0,0,0,0,0,42.426239,...,0,1,0,0,0,0,1,-83.238259,0,0
285361,10.0,0,0,0,0,0,0,0,0,42.426239,...,0,1,0,0,0,0,1,-83.238259,0,0
285338,20.0,0,0,0,0,0,0,0,0,42.309661,...,0,1,0,0,0,0,1,-83.122426,0,0
285346,10.0,0,0,0,0,0,0,0,0,42.308830,...,0,1,0,0,0,0,1,-83.121116,0,0
285345,20.0,0,0,0,0,0,0,0,0,42.308830,...,0,1,0,0,0,0,1,-83.121116,0,0
285347,5.0,0,0,0,0,0,0,0,0,42.308830,...,0,1,0,0,0,0,1,-83.121116,0,0
285342,0.0,0,0,0,0,0,0,0,0,42.313314,...,0,0,0,0,0,0,1,-83.108636,0,0
285530,100.0,0,0,0,0,0,0,0,0,42.261245,...,0,1,0,0,0,0,1,-83.160878,0,0


In [5]:
clf = MLPClassifier(hidden_layer_sizes = [100, 10], alpha = 5,
                   random_state = 0, solver='lbfgs', verbose=0)
#     clf = DecisionTreeClassifier()
clf.fit(X_train_scaled, y_train)

y_prob = clf.predict_proba(X_test_scaled)
test_proba = y_prob[:,1]

test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
test_df['compliance'] = test_proba
test_df.set_index('ticket_id', inplace=True)

test_df.compliance

ticket_id
284932    0.060149
285362    0.012152
285361    0.051423
285338    0.062048
285346    0.072370
285345    0.062057
285347    0.078285
285342    0.608303
285530    0.007981
284989    0.052735
285344    0.078517
285343    0.017502
285340    0.017394
285341    0.078093
285349    0.063376
285348    0.059912
284991    0.121659
285532    0.040909
285406    0.030497
285001    0.021044
285006    0.009205
285405    0.032155
285337    0.008215
285496    0.070827
285497    0.056187
285378    0.032485
285589    0.018384
285585    0.044167
285501    0.065654
285581    0.012029
            ...   
376367    0.010999
376366    0.030191
376362    0.214876
376363    0.251660
376365    0.010999
376364    0.030191
376228    0.037314
376265    0.030304
376286    0.227296
376320    0.034853
376314    0.034703
376327    0.294094
376385    0.272286
376435    0.732752
376370    0.594378
376434    0.059357
376459    0.063592
376478    0.000364
376473    0.034846
376484    0.030436
376482    0.023228
37

In [6]:
y_prob

array([[0.93985052, 0.06014948],
       [0.98784759, 0.01215241],
       [0.94857678, 0.05142322],
       ...,
       [0.94084868, 0.05915132],
       [0.9408483 , 0.0591517 ],
       [0.91672667, 0.08327333]])

In [12]:
print(X_train.shape)
X_train.head()

(159653, 74)


,late_fee,state_PR,state_HI,state_GA,state_PA,state_MD,state_UT,state_TX,state_DC,lat,...,state_IA,disposition_Responsible by Default,state_DE,state_MA,state_OH,agency_name_Detroit Police Department,agency_name_Department of Public Works,lon,state_AL,state_ID
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,25.0,0,0,0,0,0,0,0,0,42.390729,...,0,1,0,0,0,0,0,-83.124268,0,0
27586,75.0,0,0,0,0,0,0,0,0,42.326937,...,0,0,0,0,0,0,0,-83.135118,0,0
22046,25.0,0,0,0,0,0,0,0,0,42.145257,...,0,1,0,0,0,0,0,-83.208233,0,0
18738,75.0,0,0,0,0,0,0,0,0,42.433466,...,0,1,0,0,0,0,0,-83.023493,0,0
18735,10.0,0,0,0,0,0,0,0,0,42.388641,...,0,1,0,0,0,0,0,-83.037858,0,0


In [13]:
print(y_train.shape)
y_train.head()

(159653,)


ticket_id
22056    0.0
27586    1.0
22046    0.0
18738    0.0
18735    0.0
Name: compliance, dtype: float64

In [14]:
print(X_test.shape)
X_test.head()

(61001, 74)


,late_fee,state_PR,state_HI,state_GA,state_PA,state_MD,state_UT,state_TX,state_DC,lat,...,state_IA,disposition_Responsible by Default,state_DE,state_MA,state_OH,agency_name_Detroit Police Department,agency_name_Department of Public Works,lon,state_AL,state_ID
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,20.0,0,0,0,0,0,0,0,0,42.407581,...,0,1,0,0,0,0,1,-82.986642,0,0
285362,100.0,0,0,0,0,0,0,0,0,42.426239,...,0,1,0,0,0,0,1,-83.238259,0,0
285361,10.0,0,0,0,0,0,0,0,0,42.426239,...,0,1,0,0,0,0,1,-83.238259,0,0
285338,20.0,0,0,0,0,0,0,0,0,42.309661,...,0,1,0,0,0,0,1,-83.122426,0,0
285346,10.0,0,0,0,0,0,0,0,0,42.308830,...,0,1,0,0,0,0,1,-83.121116,0,0
